In [1]:
import csv
import numpy as np
import pandas as pd
import re
from gensim.models import KeyedVectors
import morfeusz2
from Levenshtein import distance
from collections import Counter
from sklearn.cluster import DBSCAN
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm  
import os.path
import json
df_org = pd.read_excel('../shumee_mckinsey -Aktualizacja 01.03.xlsx', index_col=None, engine='openpyxl')

In [2]:
def finder(phrase):
    return (lambda product_name : str(product_name).find(phrase))
df = df_org[df_org['ID zamówienia'].notna()] # usunięcie wpisów gdzie ID zamówienia == NaN
df = df[df['Nazwa produktu'] != 'Przesyłka pobraniowa']
df = df[df['Nazwa produktu'].map(finder("Kod rabatowy")) == -1] # usuniecie kodow rabatowych
df = df[df['Miasto'] != 'test']
df = df[df['Miasto'] != 'Test']
df = df[df['Miasto'] != 'TEST']
df = df[df['Miasto'] != 'ssss']
df = df[df['Miasto'] != 'asd']
df = df[df['Miasto'] != 'asdasd']
df = df[df['Miasto'] != 'dsss']
df = df[df['Nazwa produktu'].map(finder("TESTOWY PRODUKT")) == -1]
df = df[df['Nazwa produktu'].map(finder("PRODUKT TESTOWY")) == -1]
df = df[df['Cena'] >= 0] # usuniecie kodow rabatowych
# akceptuję produktu z ceną 0 zł, ponieważ są to produkty zamówione osobiście lub też telefonicznie

In [3]:
#pd.set_option("display.max_rows", 1000, "display.max_columns", None)
#df.loc[df['Cena'] < 0.1].head(100)
dfPL = df[df['Kraj'] == 'PL']
productsOrg = dfPL['Nazwa produktu'].unique()
products = {}
for i in range(len(productsOrg)):
    products[productsOrg[i]] = [i]

In [4]:
regex = re.compile('[^a-zA-Złóżźśęąćń]')
remove_list = list(map(lambda x:regex.sub('', x.lower()), ['shumee', 'wallart', 'szt', 'szt', 'PEPCO', 'UMBRA', 'KINGHOFF', 'TXM', 'INVICTA',
       'FMD', 'CZYSTA', 'CZYSTE',
       'MODESTO', 'DO', 'HI', 'WOW!', 'KETER',
       'PALMIYE', 'TAI', 'NEW', 'FOCUS', 'NILOE', 'FERM', 'KASSEL',
       'PLEXO55', 'HELMO', 'SISTENA', 'VALENA',
       'EMAS', 'KLAUSBERG',
       'ACTONA', 'CZYSTY', 'SIEMENS', 'PODWÓJNY', 'STRATOS',
       '\ufeffPEPCO',
       'MOSAIC',
       'TROL', 'CELIANE', 'MOVIT', 'ABIZOL',
       'VIGA', 'KARRE', 'STR',
       'SLM', 'EVA', 'EVA&LOLA',
       'SEDNA', 'DUŻA', 'KPL',
       'NOWOCZESNY', 'PERVOI,',
       'OKRĄGŁY', 'PROSTOKĄTNE', 
       'FUNKCJONALNE', 'VOLTRONIC', 'VOLANTE', 'NOWOCZESNA',
       'UNIWERSALNY', 'NATO', 'TRIXIE', 'DIVERO',      
       'L.O.L.', 'BTO-22', 'SMOBY', 'SENTIA',
       '"EVA&LOLA', "CHILDREN'S", 'STEP2', 'INDUSTRIAL',
       'CARLA', 'ELETTROCANALI', 'AKCENT', 'PURO',
       'AK', 'BAS_', 'ITP', 'GLOBOBAR', 'XQ',
       'DSG', 'BERG', 'PRZYPALENIA', 'DRITTO',
       'TON', 'CZARNA',
       'DURO', 'WASSERKESSEL', 'WINDFORCE',
       'SPAMEL', 'EATON', 'KOMPLET',
       'KPL.', 'UMBRA-',
       'PROWERK', 'SENA', 'FLEXI',
       'SKIN79', 'SANN', 'BIG',
       'MINI',
       'FEBER', 'ACTIVE', 'EXTRA', 'PROFESIONALNA',
       'SCHNEIDER', 'ELMERS', 'LINGLONG', 'DICKIE',
       'FEDERAL', 'NUTRA',
       'FITMIN', 'CERTECH', 'SABUNOL', 'NESTOR', 'VITAKRAFT',
       'PROMOCJA', 'ECO',
       'OUTLET', 'HASBRO',
       '4DOGS', 'BASKERVILLE', 'BAHCO', 'EVERLAST', 'KS',
       'YATO', 'BURG-WÄCHTER', 'LONSDALE', 'SCHÜTTE', 'BRILLIANT',
       'VOREL', 'WANDA', 'RIDDER', 'OVERZOO', 'POWER', 'BMW', 'ZOLUX',
       'FALK', 'WIO-MAR', 'FP', 'BEAPHAR', 'HEX', 'S3092',
       'BE1862', 'AS002', 'L8015',
       'TOP', 'ADBI', 'BRIT', 'WADER', 'CHIPSI',
       'APPLAWS', 'PET',
       'BSX', 'ON', 'OTEO', 'SONATA', 'XXL',
       'S86047', 'FRESH', 'VITAPOL', 'FISHER',
       'DR', 'KARO',
       'ULTIMAX', 'PIATNIK', 'CRSM', 'TOITOYS', 'IMPRESJA',
       'KWAD', 'NATURAL', 'HAM-STAKE', 'CRM',
       'NF4575', 'OASY', 'KAPPA',
       'ASFORA', 'AS', 'SIRIUS',
       'FALA', 'BABY', 'FORIX', 'BOB', 'P-51D', 'F&F', 'FONDUE',
       'HL776', 'UNIWERSALNA', 'SIMBA',
       'VACO', 'LOVATO', 'AQUANT', 'KOS66',
       'ECOTOYS', 'DUPLO',
       'HAPPY',
       'TEKNO', 'MADERA',
       'WALKIE', 'RODZINA',
       'PICTUREKA', 'YKA',
       'SA-1993', 'TY', 'DC',
       'SOKY', 'TERA', 'MICCO', 'SAVIO',
       'TG-3468', 'B360N', 'UW', 'NIEBEZPIECZEŃSTWO',
       'KX-TGC212', 'SX1888', 'NIKE',
       'METEOR', 'BD-R', 'FOTE', 'WAMOD', 'GP', '2D',
       'SA-209671', 'SES', 'FLAMINGO', 'ZMYSŁOWY',
       'SPOKEY', 'TL11149004', 'SELECT', 'MZ', 'HIP', 'VMF',
       'FAVOUR', 'GLOBO', 'M62998_1', 'FLAMINGO,', 'PUS',
       'D&D', 'YAKIMA', 'DUTCH', 'FC', 'ESPADRYLE',
       'GINO', 'ELEGANCKI', 'MONDO',
       'HOBBY', 'MDM', 'MINECRAFT', 'Z',
       'RAVEN.', 'ITALERI', 'FIFA', 'PIKMI', 'I',
       'CLEMENTONI', 'MEHANO', 'U.S.',
       'ADDO', 'REVELL', 'DWUSTOPNIOWA', 'C.BLANCHE',
       'PSI', 'DINOTRUX', 'VALLEJO', 'velda', 'xcm', 'versace'
       'GEL-A-PEEL', 'WYPRZEDAŻ', 'CR7', 'V', 'brder', 'mannesmann', 'gratis', 'little', 'tikes', 'nature', 'pro', 'evo', 'rcr', 'proplus']))
             
mapper = {"LEGO" : "klocki", 'lego' : 'klocki', 'DREWEX-POŚCIEL' : 'pościel', "DDR3" : 'pamięć ram'}

to_split = [' ', ',', '.', '-', '_', '!', '+', '*', '&']
polish_chars_mapper = {'ę':'e', 'ó':'o', 'ł':'l', 'ś':'s', 'ą':'a', 'ż':'z', 'ź':'z', 'ć':'c', 'ń':'n'}

def remove_polish_chars(word):
    return "".join(list(map(lambda x: polish_chars_mapper[x] if x in polish_chars_mapper else x, word)))

def prepare_name_products (product):
    product = product.lower()
    for i in to_split:
        product = product.replace(i, ' ')
    product = list(map(lambda x: regex.sub('', x), product.split(' ')))
    #print (product)
    product = [i for i in product if i not in remove_list]
    product = list(map(lambda x: mapper[x] if x in mapper else x, product))
    #print(product)
    result = []
    for i in product:
        if len(i) > 2:
            result.append(i)
    return " ".join(result)

def map_and_unique(function, collection):
    unique_collection = {}
    for k in collection:
        new_k = function(k)
        if new_k not in unique_collection:
            unique_collection[new_k] = []
        unique_collection[new_k] += collection[k]
    return unique_collection

unique_products = map_and_unique(prepare_name_products, products)
print (len(unique_products))

29297


In [5]:
morfeusz = morfeusz2.Morfeusz()

def get_base_name(name):
    analysis = morfeusz.analyse(name)
    pos = -1
    results = []
    for i, j, (token, base, *_) in analysis:

        # pomijam wszystkie interpretacje słowa poza pierwszym
        if pos == i:
            continue
        else:
            results.append(base.split(':')[0])  # morfeusz kończy formę bazową słowa znakiem ":"
            pos = i

    return ' '.join(results)

In [6]:
# tworzę listę nazw ich odpowiedników z formami podstawowymi
tokenized_products = map_and_unique(get_base_name, unique_products)
tokenized_products = map_and_unique((lambda x: x.lower()), tokenized_products)
print (len(tokenized_products))
tokenized_products

28833


{'ramka clip srebrny': [0, 408, 10091],
 'fontanna stalić nierdzewny basen ogrodowy': [1, 9437],
 'krzesło biurowy pomarańczowy sztuczny': [2],
 'parawan panelowy biały': [3,
  130,
  786,
  932,
  1570,
  2340,
  3962,
  4730,
  10898,
  10985,
  11256],
 'półka ścienny biały dąb sonoma płyta wiórowy': [4],
 'regał książka biały płyta wiórowy': [5, 9275, 16024, 21366],
 'wieszak ubranie subway multi szary': [6],
 'przypinka zapiąć akryl sztuk': [7],
 'biurko kolor biały naturalny': [8],
 'dywanik sztuczny futro królik ciemnoszary': [9, 17758],
 'chwytak kroić cebula': [10],
 'stojak but teleskopowy drążek aluminiowy': [11],
 'karty czarny piotruś mix super man': [12],
 'puzzle element bingo': [13, 3497, 10102],
 'puzzle element myszka mickey': [14, 9425, 10099],
 'puzzle element patrol': [15, 3498, 9426, 10100],
 'puzzle element świnka pepa': [16, 4706],
 'kosz gabionowy stalić kształt litera': [17, 18858],
 'ściana gabionowy pokrywa galwanizować stal': [18,
  1107,
  4359,
  24387,
 

In [7]:
colors = list(map(remove_polish_chars, ['black', 'color', 'antracytow', 'bialo', 'jasnoszary', 'silver', 'melanz', 'white', 'ciemno', 'ciemny', 'fioletowo', 'niebiesko', 'jasny', 'roz', 'szaro', 'biel', 'mat', 'braz', 'ciemnobezowy', 'grafitowyowy', 'bezow', 'cza', 'czarno', 'pomarancza', 'zielone', 'szarosc', 'czerwona', 'ciemnoszary', 'akwamaryna', 'alabastrowy', 'amarantowy', 'ametystowy', 'antracytowy', 'atramentowy', 'bahama yellow', 'bananowy', 'beżowy', 'biały', 'biskupi', 'błękit', 'bordowy', 'bordo', 'brązowy', 'brunatny', 'brzoskwiniowy', 'buraczkowy', 'burgund', 'bursztynowy', 'bury', 'ceglasty', 'chabrowy', 'chamois', 'cielisty', 'cukierkowy', 'cyjan', 'cyklamen', 'cynamonowy', 'cynobrowy', 'cytrynowy', 'czarny', 'czerwień', 'czerwony', 'eozyna', 'feldgrau', 'fioletowy', 'fiołkowy', 'fuksja', 'grafitowy', 'granatowy', 'grynszpan', 'hebanowy', 'herbaciany', 'indygo', 'jagodowy', 'jasnożółty', 'jaśminowy', 'kanarkowy', 'kardynalski', 'karmazynowy', 'karminowy', 'kasztanowy', 'kawowy', 'khaki', 'kobaltowy', 'koniakowy', 'koralowy', 'krwisty', 'lawendowy', 'lazurowy', 'lila', 'liliowy', 'limonkowy', 'łososiowy', 'magenta', 'mahoń', 'mahoniowy', 'majtkowy', 'makowy', 'malachitowy', 'malinowy', 'marchewkowy', 'marengo', 'miedziany', 'miętowy', 'minia', 'miodowy', 'mleczny', 'modry', 'morelowy', 'morski', 'mosiądzowy', 'mysi', 'niebieski', 'niebieskofioletowy', 'oberżynowy', 'ochra', 'oliwkowy', 'oranż', 'orzechowy', 'palisander', 'patynowy', 'pąsowy', 'perłowy', 'piaskowy', 'pistacjowy', 'platynowy', 'płowy', 'pomarańczowy', 'popielaty', 'porcelanowy', 'poziomkowy', 'purpurowy', 'rdzawy', 'różowy', 'rubinowy', 'rudy', 'sadza angielska', 'seledynowy', 'sepia', 'siarkowy', 'siny', 'siwy', 'słomkowy', 'smolisty', 'spiżowy', 'srebrny', 'stalowy', 'szafirowy', 'szafranowy', 'szary', 'szkarłatny', 'szmaragdowy', 'śliwkowy', 'świniowy', 'tabaczkowy', 'tango', 'truskawkowy', 'turkusowy', 'tycjan', 'ugier', 'ultramaryna', 'umbra', 'winny', 'wiśniowy', 'woskowy', 'wrzosowy', 'zieleń', 'zielony', 'złocisty', 'złoty', 'żółty']))
materials = list(map(remove_polish_chars, ['drewniany', 'tekowy', 'akacja', 'eukaliptusowy', 'zelowy', 'sosnowe', 'aluminiu', 'antracyt', 'eukaliptus', 'debowy', 'topolowy', 'sosna', 'orzech', 'mdf', 'metalowy', 'bukowy', 'PVC', 'aluminium', 'mango', 'plastikowy', 'metal', 'zelazo', 'bambusowy', 'poliweglan', 'stones', 'galwanizowac', 'stalic', 'aluminiowy', 'dab', 'akacjowy', 'papierowy', 'papierowy', 'stal', 'betonowy', 'nierdzewny', 'sztuczny']))
others = ['vintage', 'wzor', 'loreal', 'experta', 'kolor', 'kolorowy', 'profi', 'audi', 'jeep' 'wrangler', 'willys', 'terenowy',
          'land', 'rover', 'defender', 'chicago', 'sta', 'rozm', 'naturalny', 'gxp', 'lity', 'eko', 'eco', 'oxford', 'sztuk',
          'xxcm', 'made', 'italia', 'german', 'soviet', 'wysoki', 'niski', 'lekki', 'luksusowy', 'mala', 'maly', 'esschert', 'design', 'bruder']
def color_material_other_remover (product):
    product = product.split(' ')
    result = []
    for i in product:
        if not i in colors and not i in materials and not i in others:
            result.append(i)
    return " ".join(result)

def words3(product):
    return " ".join(product.split(' ')[:3])

In [8]:
tokenized_productsWPC = map_and_unique(remove_polish_chars, tokenized_products)
tokenized_productsWC = map_and_unique(color_material_other_remover, tokenized_productsWPC)
tokenized_productsWC = map_and_unique(words3, tokenized_productsWC)
print (len(tokenized_productsWC))
tokenized_productsWC

16339


{'ramka clip': [0, 408, 10091, 4994, 11465],
 'fontanna basen ogrodowy': [1, 9437],
 'krzeslo biurowy': [2],
 'parawan panelowy': [3,
  130,
  786,
  932,
  1570,
  2340,
  3962,
  4730,
  10898,
  10985,
  11256,
  129,
  2130,
  2587,
  3323,
  4993,
  7151,
  11145,
  12521,
  12848,
  23896,
  374,
  1671,
  3218,
  3574,
  3655,
  3689,
  11489,
  17767,
  22775,
  26386,
  29130,
  383,
  1110,
  1489,
  1563,
  1595,
  2480,
  3680,
  4722,
  4759,
  8310,
  11483,
  11564,
  12151,
  16139,
  18605,
  29546,
  1409,
  2382,
  4635,
  15012,
  7938,
  18147,
  27967],
 'polka scienny sonoma': [4, 1598, 3210, 9282, 15102, 17899, 25593],
 'regal ksiazka plyta': [5,
  9275,
  16024,
  21366,
  18505,
  24546,
  840,
  6299,
  16905,
  19528,
  23457,
  5645,
  6442,
  9439,
  15762,
  11276,
  15958,
  17795,
  27345,
  4065,
  4241,
  18236,
  18284,
  15309,
  15364,
  16336,
  17113],
 'wieszak ubranie subway': [6],
 'przypinka zapiac akryl': [7, 3503, 3504],
 'biurko': [8,
  77

In [9]:
productsDB = list(tokenized_productsWC.keys())
number_of_products = len(productsDB)
distanceMatrix = None
special_categories = [['maszynka', 'maszyna'], ["namiotowy", 'kempingowy'], ['prysznicowy', 'prysznic'], ['zewnatrz', 'zewnetrzny'], ['stojak'], ['ostrzalka', 'oselka'], ['tarka'], ['but', 'bucik']]

def products_distance(prod1, prod2):
    prod1 = list(prod1.split(' '))
    prod2 = list(prod2.split(' '))
    if (len(prod1) > len(prod2)):
        pom = prod1
        prod1 = prod2
        prod2 = pom
    if (len(prod1) == 0):
        return 2000
    summary_distance = 0
    for i in special_categories:
        first_prod_con_spec_word = ""
        second_prod_con_spec_word = ""
        for j in i:
            if j in prod1:
                first_prod_con_spec_word = i[0]
            if j in prod2:
                second_prod_con_spec_word = i[0]
        if first_prod_con_spec_word != second_prod_con_spec_word:
            return 1500
    for i in prod1:
        mn_dist = 0
        if len(i) > 0:
            for j in prod2:
                if i == j and i[-1] != 'y':
                    dst = 2
                else: 
                    dst = 1-min(1, distance(i, j)/max(len(i), len(j)))
                if (dst < 0.4):
                    dst = 0.0
                mn_dist = max(dst, mn_dist)
        summary_distance += mn_dist
    result = 2000-(summary_distance/len(prod1)*1000)
    return max(0, result)

def calculate_one_row_distance(i):
    global number_of_products, productsDB
    m = np.full(number_of_products, -1)
    m[i] = 0.0
    for j in range(i, number_of_products):
        m[j]=products_distance(productsDB[i], productsDB[j])
    return m

In [10]:
generate_new_matrix = True
path_to_save = '../distanceMatrix.npy'
if os.path.isfile(path_to_save) and generate_new_matrix == False:
    print("Reading from file")
    distanceMatrix = np.load(path_to_save)
else:
    print("Calculating distance matrix")
    num_cores = multiprocessing.cpu_count()
    distanceMatrix = np.array(Parallel(n_jobs=num_cores)(delayed(calculate_one_row_distance)(i) for i in tqdm(range(number_of_products))))
    i_lower = np.tril_indices(number_of_products, -1)
    distanceMatrix[i_lower] = distanceMatrix.T[i_lower]
    np.save(path_to_save, distanceMatrix)
print(distanceMatrix.shape)
distanceMatrix

  0%|          | 0/16339 [00:00<?, ?it/s]

Calculating distance matrix


100%|██████████| 16339/16339 [02:35<00:00, 105.02it/s] 


(16339, 16339)


array([[   0, 2000, 2000, ..., 2000, 2000, 1800],
       [2000,  333, 2000, ..., 2000, 2000, 2000],
       [2000, 2000,  500, ..., 2000, 2000, 2000],
       ...,
       [2000, 2000, 2000, ...,    0, 2000, 1866],
       [2000, 2000, 2000, ..., 2000,    0, 2000],
       [1800, 2000, 2000, ..., 1866, 2000,    0]])

In [11]:
dbscan = DBSCAN(eps=500, min_samples=2, metric='precomputed', n_jobs=1)
clustering = dbscan.fit(distanceMatrix)


In [12]:
mergedDBScanResults = {}
merged2pass = {}
labels = list(clustering.labels_)
short_name = {}
output = sorted([(i, j, []) for i, j in zip(labels, productsDB)])
for i in range(len(output)):
    tpl = list(output[i])
    tpl[2] = tokenized_productsWC[tpl[1]]
    output[i] = tpl
counter = 0
diffkeys = set()
for i, j, k in output:
    if i >= 1:
        print (i, j)
        if i in short_name:
            prd = j.split(" ")[:2]
            prdM = short_name[i].split(' ')
            if prdM[0][-1] == 'y':
                short_name[i] = " ".join(prd)
            elif len(short_name[i]) > len(j):
                short_name[i] = " ".join(prd)
        else:
            short_name[i] = j
        if i not in mergedDBScanResults:
            mergedDBScanResults[i] = []
        mergedDBScanResults[i] += k
        counter += (len(k))
    else:
        key = " ".join(j.split(' ')[:2])
#         key = j.split(' ')[0]
        diffkeys.add(key)
#         print(key)
        if key not in merged2pass:
            merged2pass[key]=[]
        merged2pass[key] += k
print(counter)
print(mergedDBScanResults)

1 fontanna basen
1 fontanna basen leda
1 fontanna basen ogrodowy
1 fontanna leda basen
1 fontanna ogrodowy
1 fontanna ogrodowy forma
1 fontanna ogrodowy ksztalt
1 fontanna ogrodowy kula
1 fontanna ogrodowy leda
1 fontanna ogrodowy nie
1 fontanna ogrodowy nier
1 fontanna ogrodowy pompka
1 fontanna wodospad basen
1 ogrodowy fontanna kulisty
1 prostokatny fontanna basen
1 wodospad fontanna basen
2 przypinka
2 przypinka zapiac akryl
2 zestawic przypinka
3 polka but stojak
3 stojak but
3 stojak but cfw
3 stojak but cztery
3 stojak but lawka
3 stojak but mwgroup
3 stojak but par
3 stojak but polka
3 stojak but teleskopowy
3 stojak but trzy
3 vidaxl stojak but
4 puzzle element ksieznichki
4 puzzle element ksiezniczka
4 puzzle element plac
4 puzzle element sniezka
4 puzzle element spiaca
4 puzzle element spider
4 puzzle element swinka
4 puzzle element zaplatac
5 sciana gabionowy
5 sciana gabionowy pokrywa
6 dwustronny zegar scienny
6 kinowy zegar scienny
6 scienny zegar ogrodowy
6 zegar scienn

47 autko skalac czolg
47 autko skalac fiat
47 autko skalac hummer
47 autko skalac jeep
47 autko skalac policja
47 autko skalac policyjny
47 autko skalac syrena
47 autko skalac wojskowy
47 autko welly bentley
47 autko welly ford
47 autko welly on
47 autko welly toyota
47 autko wojskowy
47 autko wyrzutnia
47 bburago model
47 bburago model alfa
47 bburago model bugatti
47 bburago model lamborghini
47 bburago model mercedes
47 bburago model renault
47 bentley samochod zabawka
47 blokada rozrzad samochod
47 carrera samochod wyscigowy
47 carrera samochod zabawkowy
47 carrera zabawkowy samochod
47 coupe
47 dzwig samochod wyciagarka
47 elektryczny samochod dla
47 element samochod arrinera
47 garaz
47 garaz dla kosiarka
47 garaz parking samochod
47 grac domino samochod
47 jezdzik samochod fiat
47 kamera cofac samochod
47 klocki block element
47 klocki block elementy
47 klocki budowniczy element
47 klocki city samochod
47 klocki klocki city
47 klocki mycity element
47 klocki mycity elementy
47 l

200 namiot osobowy element
200 namiot osobowy granatowo
200 namiot osobowy jasnoniebieski
200 namiot osobowy typ
200 namiot turystyczny osobowy
200 vidaxl namiot osobowy
201 rekawiczka cekin
201 rekawiczka dziewczecy
201 rekawiczka dziewczecy cekin
201 rekawiczka dziewczecy rozowo
201 rekawiczka dziewczecy zakardowy
201 rekawiczka zimowy dziewczecy
202 lustro intesi aurora
202 lustro intesi flora
202 lustro intesi flowe
203 lampka sygnalizacyjny
203 lampka sygnalizacyjny acdc
203 lampka sygnalizacyjny leda
203 lampka sygnalizacyjny mbsdb
203 lampka sygnalizacyjny mbsdk
203 lampka sygnalizacyjny mbsdm
203 lampka sygnalizacyjny mbsds
203 lampka sygnalizacyjny mbsdy
203 lampka sygnalizacyjny mbspy
204 talerz saneczkowy
204 talerz saneczkowy supernovum
205 folia lustrzany okno
205 lustrzany folia okno
206 acces meski rekawiczka
206 meski rekawiczka
206 meski rekawiczka itouch
207 ogrodowy szopa narzedzie
207 szopa ogrodowy
207 szopa ogrodowy antracy
207 szopa ogrodowy drewno
207 szopa ogro

509 opaska kablowy tabliczka
509 opaska kablowy tub
509 opaska kablowy wykrywalny
509 opaska kablowy wysokotemperaturowy
510 lampka ogrodowy leda
510 lampka stolowy leda
511 globobarek globus barek
511 globus barek globobarek
512 galazka winorosl
512 galazka winorosl czerwone
513 wspornik slupek
513 wspornik slupek ogrodowy
514 deskorolka longboard
514 deskorolka longboard palm
515 interaktywny malpka
515 interaktywny malpka malpka
516 elektryczny maszynka golenie
516 maszynka elektryczny golenie
517 gokart pedal
517 gokart pedal opona
517 trojkolowy gokart pedal
518 biustonosz angel rozmiar
518 biustonosz anita
518 biustonosz anita milk
518 biustonosz anita rozmiar
518 biustonosz anna rozmiar
518 biustonosz ashley rozmiar
518 biustonosz blanca rozmiar
518 biustonosz catia rozmiar
518 biustonosz chryzolit rozmiar
518 biustonosz clarissa rozmiar
518 biustonosz cloe rozmiar
518 biustonosz constanza rozmiar
518 biustonosz cristal rozmiar
518 biustonosz cyganeria rozmiar
518 biustonosz dor

In [13]:
print("Not clustered:", len(diffkeys))
print("Clustered by DBSCAN:", len(mergedDBScanResults))

Not clustered: 8442
Clustered by DBSCAN: 624


In [14]:
print ("Generated", len(merged2pass)+len(mergedDBScanResults), "categories")

Generated 9066 categories


In [30]:
clustered_productsId = {}
products_to_merge = ['gra', 'figi'] #['puzzle', 'wycieraczka', 'wazon', 'termos', 'dywanik', 'plafoniera', 'pizama', 'piaskownica', 'pendrive', 'pajac', 'ostrzalka', 'myszka', 'mozdzierz', 'mozaika', 'mikser', 'maskotka', 'markiza', 'majtki', 'lutownica', 'lustro', 'lutownica', 'legginsy', 'latarka', 'lampion', 'lampa', 'kurtka', 'koszula', 'koszulka', 'kostium', 'kompostownik', 'kolorowanka', 'kolyska', 'koldra', 'kolek', 'kolczyk', 'klej', 'klips', 'klocki', 'klawiatura', 'kierownica', 'karafka', 'kapielowki', 'kapec', 'kamera', 'kalosz', 'kalendarz', 'kabel', 'hulajnoga', 'hoker', 'hamak', 'grzejnik', 'grac', 'flaga', 'filizanka', 'etui', 'amperomierz', 'gitara', 'globus', 'frytkownica', 'figurka', 'figi', 'dyfuzor', 'drukarka', 'doniczka', 'dezodorant', 'bluza', 'blacha', 'apteczka', 'autko', 'bokserki', 'biustonosz', 'blender', 'bransoletka', 'brelok', 'brzeszczot', 'budzik']
for i in mergedDBScanResults:
    place = " ".join(short_name[i].split(' ')[:2])
    prod = short_name[i].split(' ')
    for k in prod:
        if k in products_to_merge:
            place = k
    if place in clustered_productsId:
        clustered_productsId[place] += mergedDBScanResults[i]
    else:
        clustered_productsId[place] = mergedDBScanResults[i]
#     clustered_productsId[short_name[i]] = mergedDBScanResults[i]
#     print ('DB: ', short_name[i], mergedDBScanResults[i])
for i in merged2pass:
    place = i
    prod = i.split(' ')
    for k in prod:
        if k in products_to_merge:
            place = k
    if place in clustered_productsId:
        clustered_productsId[place] += merged2pass[i]
    else:
        clustered_productsId[place] = merged2pass[i]
    print(place, merged2pass[i])
clustered_products = {}
last_renamer = {'figi joy':'figi', 'lustro intesi':'lustro', 'zab':'pasta zebow', 'budzik disney':'budzik', 'grac kubb':'gra planszowa', 'rekawiczka paska':'rekawiczka', 'hortensja doniczka':'sztuczna roslina', 'hoker charles':'hoker', 'lampa conic':'lampa', 'wies':'praska', 'trzyczesciowy zestawic':'zestaw mebli', 'lustro wiklinowy': 'lustro', 'podest prysznic':'prysznic ogrodowy', 'narzedzie kuchenny':'zestaw narzedzi', 'idaxl markiza':'markiza', "kinkiet" : 'lampa', 'puzzle classic':'puzzle', 'puzzle element' : 'puzzle', 'hailo kosz': 'kosz smieci', 'pokrywka stojak' : 'stojak parasol', 'umywalka matowy':'umywalka', 'umywalka marmurowy':'umywalka', 'perel zagiel': 'zagiel przeciwsloneczny', 'bluza dziewczecy':'bluza'}
for i in clustered_productsId:
    prods = []
    for j in clustered_productsId[i]:
        prods.append(productsOrg[j])
    prodName = i
    if prodName in last_renamer:
        prodName = last_renamer[prodName]
    if prodName in clustered_products:
        clustered_products[prodName] += prods
    else:
        clustered_products[prodName] = prods


 [2249, 2250, 16503, 20088, 26847, 27897, 35535, 35536, 35537, 37856, 21128, 21129, 27227, 36177]
abazur cotton [19827]
abazur ginger [26019]
abazur livigno [23506]
abazur livingo [4041]
abazur rotan [23313]
abbey kij [26221]
abbey kijek [26061]
acces chusta [9931, 9932, 13582]
acces cienki [13563]
acces czapka [13566, 13567, 13568, 13540, 13562, 13591]
acces gruby [13552, 13572]
act przycisk [5025]
activity gardna [6933, 6959]
adapter displayport [29883]
adapter displayportm [35470]
adapter dla [36036]
adapter dwukierunkowy [35592]
adapter klucz [29266]
adapter kranowa [29630]
adapter ladowac [15641]
adapter lclc [30434]
adapter micro [33554]
adapter montazowy [17021]
adapter null [29715]
adapter plc [2231]
adapter plug [7852]
adapter przejsciowka [29479]
adapter przelotka [29020]
adapter przelotowy [19426]
adapter sata [35080]
adapter spliter [2316]
adapter swiatlowodowy [30433]
adapter typ [29765]
adapter usb [36078, 21417, 28418, 36081, 36090]
adapter vesa [30102, 34017]
adapter vg

grac bingo [38762]
grac biznes [40215]
grac blok [40058]
grac boomtrix [32587]
grac bop [30683]
grac cardinal [33579]
grac catan [28463, 40373]
grac czolko [32381]
grac dobble [39967]
grac domino [30747]
grac edukacyjny [9875, 21950, 21883, 24074]
grac elektroniczny [25048]
grac flirt [34280]
grac grzybobranie [8782, 9630, 11022]
grac headu [32239]
grac horror [31201]
grac just [36681]
grac karaka [31174]
grac karty [37947]
grac koniec [40002]
grac krokodyl [69, 25111]
grac kwartet [34279, 34281]
grac list [29676]
grac looping [30358]
grac lowic [40432, 336, 1998, 6236, 7858, 23835]
grac mafia [39973]
grac memory [31900]
grac moj [40053]
grac monopoly [30588, 26586, 35829]
grac paleontologia [40885]
grac pandemic [39759]
grac panstwo [32968]
grac paszczak [29692]
grac planszowy [7587, 8926, 31029, 7554, 28360, 10308, 30759, 7387, 18532]
grac plaster [604, 4893, 11037]
grac plenerowy [19753]
grac podrozny [35866]
grac rach [3030]
grac runebound [40400]
grac ryzyko [40409]
grac sen [4011

lozeczko gwiazdka [33039]
lozeczko kuba [33041]
lozeczko lisek [27416]
lozeczko mocca [35055]
lozeczko turystyczny [20772]
lozysko przedni [30391]
lucy szampon [29115, 28910]
luk bloczkowy [39863]
luk masaz [40114]
luk refleksyjny [14128, 36393]
luk rozany [25100, 4939, 23409]
luk swiateczny [10259]
lunchbox przegroda [34444]
lund narzedzie [37211]
lusterko kosmetyczny [24495, 7136, 9146]
lusterko okragly [22181, 25960, 20597, 25533, 29132]
lusterko stojacy [23503, 24271, 25752, 28484, 25462, 23786, 25355, 25405, 25406, 26522, 23927, 26655, 21317, 29768]
lustro arabesque [24816]
lustro gotycki [17157]
lustro innovagoods [38793]
lustro intesi [24384, 24714, 2821, 5147, 12598, 17008, 8505, 13466, 17258, 13204, 16606, 27143, 37366, 37845, 711, 16060]
lustro jones [22810]
lustro lazienka [11700, 16376, 31963]
lustro lazienkowy [6703, 10120, 13286, 28441, 11170, 21016, 21904, 18212, 41502, 28526]
lustro luma [19676]
lustro magazynowy [456, 9026]
lustro octano [27551]
lustro ogrodowy [19952,

pluszak national [34664]
pluszowy brachiozaur [41314]
pluszowy brontozaur [7089, 31278]
pluszowy buldog [39300]
pluszowy dywanik [3828, 12787, 22874, 31480]
pluszowy hipopotam [7580, 7603]
pluszowy jamnik [4450, 36589]
pluszowy konik [4263, 7294, 8039, 8502, 34785, 34852]
pluszowy krowa [3997, 14647, 36884]
pluszowy krowka [17140]
pluszowy labrador [31072]
pluszowy leopard [2670]
pluszowy maskotka [4278, 8776, 32390, 5429, 6979, 41070]
pluszowy mis [34133, 754, 5497, 17399]
pluszowy mops [2815]
pluszowy niedzwiedz [35306]
pluszowy osiolek [7013]
pluszowy owca [16401]
pluszowy panda [18210]
pluszowy pantera [2387]
pluszowy przytulanka [8583]
pluszowy rekin [4069, 20929, 7459]
pluszowy rottweiler [8411]
pluszowy serce [9019]
pluszowy slon [7321]
pluszowy zaba [40175]
pluszowy zyrafa [166, 4585, 10542]
plyn dezynfekcja [17089, 26125]
plyn plukanie [24718]
plyn zmywarka [40117, 40687]
plynny podklad [40382, 19682, 36652, 29592, 36730, 40353, 34467]
plyta dvd [29070, 30813]
plyta gazowy [11

spodniepanostyle poliester [26704]
sportowy aparat [39536]
sportowy opaska [40683]
sports maska [13507]
sprawdzian matematyka [39636]
sprawdzian szostoklasista [39760]
spray depilacja [40611, 39438]
spray golebi [25397]
spray kuna [32332]
spray modelowac [37999]
spray okular [28098]
spray opalac [37042, 41001, 22362, 35932, 28476, 40447, 40712, 38900, 40319, 40485, 40519, 40713, 40719]
spray rozswietlac [40537]
spray wygladzac [35727, 39611]
sprezyna amortyzator [30061]
sprezyna hydrauliczny [5906]
spryskiwacz spray [40180]
sprytny inteligentny [33332]
sprzet studia [14109, 40895, 25937, 21730, 29826]
sredni walizka [20350, 28806]
srodki osuszac [34660]
sruba dwugwintowy [32139, 32140, 32141]
sruba leb [31328, 31784]
sruba napedowy [41083]
sruba rzymski [33829]
sruba typ [32547]
stabilizator lodz [12951]
stabilizatorywsporniki [35846]
stacja deratyzacyjny [30987]
stacja pogodowy [4639, 17257]
stacja prasowac [39415]
stacja projektowy [7311]
stacjonarny rower [17308, 34106]
standardowy 

wysiegnik wsz [32484]
wystrzalowy okulary [35714]
wysuwac kosz [971, 3660, 4257, 5359, 6297, 17352, 28861, 28862, 39503, 3355, 8318]
wysuwac markiza [926, 1297, 1361, 2916, 3579, 4350, 10345, 12749, 12789, 14420, 15099, 15334, 17039, 17700, 18252, 18658, 18860, 19001, 19005, 19391, 19569, 19730, 20120, 20721, 20990, 21702, 21822, 23519, 23522, 24580, 27858, 3276, 22596, 22816, 23899, 26445, 28293, 4089, 4191, 15615, 22632, 26517, 27270, 27734, 25672, 1413, 4882, 5297, 17427, 21479, 8839, 9027, 9034, 19610, 22034, 23296, 11848, 16076, 25621, 19293]
wysuwac stojak [16702]
wysuwac taca [32570]
wyswietlacz tekstowy [30903]
wyszczuplac polhalka [39691]
wytaczadlo trzpien [1418, 10048]
wytrzymac klatka [7119, 11078]
wyzwalacz podnapieciowy [13692]
wyzwalacz wzrostowy [33846, 1392]
wzmacniacz antenowy [28946]
wzmacniacz dzwiek [8149]
xcomfort adapter [7040]
xcomfort bezprzewodowy [6992, 32295]
xcomfort klawisz [31970]
xiaomi roidmi [5843]
xqmax darts [39561, 36849, 40755, 39588, 40935, 41076,

intex materac [15023, 15024, 15413, 19301, 24422, 26607, 33074, 33925, 35686]
intex podgrzewacz [17435, 36165]
japonka damski [23949]
jednoosobowy hustawka [17136, 19636, 38755]
jednorozec [257, 577]
jednorozec biegun [16652]
jednorozec medium [10089, 13978]
jeggins damski [9839]
jegginsy dama [9840, 9841, 9842]
jezdzik skuter [28077]
jilong materac [21590]
joystick ramka [6923]
kanal elastyczny [31720]
kapelusz damski [19363, 24458, 8923]
kapielowki bokserki [20095]
klamerka zamykac [2429, 10917]
klapka damski [11591]
klasyczny damski [18409]
klasyczny zamek [11287]
klatka zadaszyc [25574]
klucz zapasowy [4386, 6058, 11895, 11896, 29382]
koc [3786, 8400, 20575, 23390, 24004]
koc dzieciecy [40866]
koc elektryczny [1327, 10299, 31660, 41013, 4999, 5672, 11758, 38998]
koc flanela [7403, 11206, 18397, 19249]
koc flanelowy [27464]
koc mikrofibra [746, 10137, 13893, 748, 2704, 13067, 13449, 17063, 13953, 19992, 23441, 23523, 747, 12148, 9907, 9908, 9909, 13122, 17560, 16847]
koc pastello [2

pojemnik trzy [6558]
pojemnik warsztatowy [27639]
pojemnik zywnosc [24075, 25192]
pokrowiec dwa [9030]
pokrowiec dwuosobowy [10097, 16756]
pokrowiec fotel [7959]
pokrowiec grill [23316, 36683, 10518, 17702]
pokrowiec hustawka [3119, 11832, 12522, 4758, 12776, 7904, 13822, 14437, 14533]
pokrowiec kosiarka [23212]
pokrowiec krzeslo [8668, 30713, 39141, 40502, 7698, 149, 3708, 7042, 9397, 16082, 9493, 9681, 18244, 8688, 34843, 15944, 21711]
pokrowiec lezak [13173, 14499, 36738, 5467, 8277, 35058]
pokrowiec ogrodowy [17321]
pokrowiec oparcie [8466, 30663]
pokrowiec parasol [4372, 26997, 8430, 8452, 22010, 22963, 23227, 12475, 14853, 16057, 16334]
pokrowiec poduszka [35854, 9402, 10333, 12606, 13878, 15874, 24377]
pokrowiec prostokatny [8114, 11283, 16058, 23784]
pokrowiec sofa [5759, 22501, 7961]
pokrowiec stol [22902, 1298, 3688, 12911, 16056]
pokrowiec stolik [19672, 24322, 24684]
pokrowiec zamek [35639]
polarowy koc [33037]
polka dwa [40686]
polka kolka [15393, 19191, 25485, 36297]
polk

szczypce grill [36577]
szczypce klucz [31652]
szczypce kuchenny [32923]
szczypce marble [13932, 17613]
szczypce montaz [26432]
szczypce nierdzewneplastik [1269, 1839, 12551]
szczypce nierdzplastik [18034, 18035]
szczypce ogrodowy [39908]
szczypce pierscienic [36400]
szczypce silikonowy [3241]
szczypce sitko [3014]
szczypce zaciskowy [37450]
szeroki spodni [36120, 38954]
szesciokatny namiot [20242]
szezlong kremowy [12629]
szezlong masowac [4066, 16197, 13356, 17507, 28337, 24392, 13838, 13845, 17654, 19123, 21875, 16765, 34336]
szezlong poduszka [28766, 40833, 5148, 5356, 12442, 20035, 10062, 19755, 14406, 25618]
szezlong skora [10816, 32090, 25285, 14962, 20033, 16149, 27272, 31937, 32474, 27620, 30994, 17413, 23676]
szklany biurko [27009, 2899]
szklany pojemnik [434, 1552, 14495, 32292]
szklany polka [6834, 9730, 6277, 15614, 24655, 32176, 13009, 32336, 33061, 41361]
szklany stolik [1776]
szklarnia [402, 3336, 5804, 14436, 25451, 5433, 11235, 20987, 21534, 28405, 28560, 29188, 30342,

In [31]:
print(len(clustered_products))
clustered_products

8977


{'fontanna basen': ['Fontanna do basenu, stal nierdzewna, 64 x 30 x 52 cm, srebrna',
  'Fontanna do basenu, stal nierdzewna, 50 x 30 x 90 cm, srebrna',
  'Fontanna do basenu, stal nierdzewna, 64 x 30 x 52',
  'Fontanna do basenu, stal nierdzewna, 30 x 9 x 26 c',
  'Fontanna do basenu, stal nierdzewna, 50 x 30 x 53',
  'Fontanna do basenu, stal nierdzewna, 45 x 30 x 65',
  'Fontanna do basenu LED, 15,8x11,8x24 cm',
  'Fontanna do basenu LED, 22,5x17x25,5 cm',
  'Fontanna do basenu LED, 13,5x13,5x18,5 cm',
  'Fontanna do basenu z LED, stal nierdzewna, 30x60x4',
  'Fontanna ze stali nierdzewnej do basenu ogrodowego',
  'Fontanna ze stali nierdzewnej do basenu ogrodowego, 45x30x60 cm',
  'Fontanna LED do basenu PERVOI, biała, 43x31 cm',
  'Fontanna LED do basenu od Intex, PP',
  ' Fontanna ogrodowa, srebrna, 48x34x88 cm, stal nierdzewna',
  'Fontanna ogrodowa, srebrna, 48x34x88 cm, stal nierdzewna',
  'Fontanna ogrodowa, srebrna, 48x34x123 cm, stal nierdzewna',
  'Fontanna ogrodowa, srebrn

In [32]:
a = 'word'
a[1:2]

'o'

In [33]:
import operator
clustered_products_fixed_names = {}

# def distanceLav (a, b):
#     a = remove_polish_chars(a.lower())
#     b = remove_polish_chars(b.lower())
#     return distance(a, b)

def prepWord(a):
    return remove_polish_chars(a.lower())

def get_full_name(a, b, prod):
    prod = prod.split(' ')
    mna = 100
    mnb = 100
    aa = 0
    bb = 0
    for i in range(len(prod)):
        w = prepWord(prod[i])
        disAW = distance(a, w)
        disBW = distance(b, w)
        if disAW < mna:
            mna = disAW
            aa = i
        if disBW < mnb:
            mnb = disBW
            bb = i
    result = " ".join(prod[min(aa, bb):(max(aa, bb)+1)])
    result_sim = prepWord(result)
    for i in to_split:
        result_sim = result_sim.replace(i, ' ')
    result_sim = list(map(lambda x: regex.sub('', x), result_sim.split(' ')))
    resultS = []
    for i in result_sim:
        if len(i) > 2:
            resultS.append(i)
    return (" ".join(resultS), result)

in_prod_to_remove = ['cm', 'lr', 'szt', 'l', 'm']

def insert_productHlp(prodN, products):
    prod = []
    prodN = prodN.split(' ')
    for i in prodN:
        if i not in in_prod_to_remove:
            prod.append(i)
    prodN = " ".join(prod)
    if prodN in clustered_products_fixed_names:
        clustered_products_fixed_names[prodN] += products
    else:
        clustered_products_fixed_names[prodN] = products
        

        
def insert_product(counter, i, clustered_prodI, full_name):
    if len(counter) == 0:
        prod = i.lower()
        for k in to_split:
            prod = prod.replace(k, ' ')
        prod = list(map(lambda x: regex.sub('', x), prod.split(' ')))
        insert_productHlp(" ".join(prod), clustered_products[i])
    else:
        new_word = full_name[max(counter.items(), key=operator.itemgetter(1))[0]]
        new_word = new_word.lower()
        new_word = new_word.split(',')[0]
        for k in to_split:
            new_word = new_word.replace(k, ' ')
        new_word = list(map(lambda x: regex.sub('', x), new_word.split(' ')))
        wordToAdd = []
        for k in range(len(new_word)-1):
            if k == 0:
                wordToAdd.append(new_word[k])
            else:
                if len(new_word[k])<5 and len(new_word[k])>1:
                    wordToAdd.append(new_word[k])
        wordToAdd.append(new_word[-1])
        #print(i, new_word, wordToAdd)
        insert_productHlp(" ".join(wordToAdd), clustered_products[i])
    
for i in clustered_products:
    prodName = i.split(' ')[:2]
    counter = {}
    full_name = {}
    if len(prodName) == 1:
        for j in clustered_products[i]:
            for k in j.split(' '):
                kk = prepWord(k)
                if distance(kk, i) < 3:
                    if kk in counter:
                        counter[kk] += 1
                    else:
                        counter[kk] = 1
                        full_name[kk] = k
        insert_product(counter, i, clustered_products[i], full_name)
    elif len(prodName) == 2:
        for j in clustered_products[i]:
            #print (prodName[0], ":", prodName[1], ":", j)
            sim, full = get_full_name(prodName[0], prodName[1], j)
            #print (sim, ":", full)
            if len(sim) > 2:
                if sim in counter:
                    counter[sim] += 1
                else:
                    counter[sim] = 1
                    full_name[sim] = full
        insert_product(counter, i, clustered_products[i], full_name)
        

In [34]:
clustered_products_fixed_names.keys()

dict_keys(['fontanna do basenu', 'przypinki', 'stojak na buty', 'puzzle', 'ściana gabionowa', 'zegar ścienny', 'dozownik mydła', 'wklad do szuflady', 'dywan', 'pokrywa na basen', 'roleta', 'świetlówka', 'babeczek', 'wieszaczki samoprzylepne', 'tarka', 'lampa', 'obrzeże ogrodowe', 'kosz na śmieci', 'stojak na parasole', 'wykładzina targowa', 'uchwyt do telewizora', 'dywanik łazienkowy', 'noży', 'korkociąg', 'świeca szkle', 'karabin pistolet', 'pudełko składane', 'blacha do ciasta', 'markery permanentne', 'umywalka', 'tablet graficzny', 'latarenka', 'siatka na siano', 'patelnia', 'podstawa pod', 'żagiel przeciwsłoneczny', 'nakładki na schody', 'bluza', 'deska kuchenna', 'kabina prysznicowa', 'słoiki', 'dywanik tekstylny', 'drzwi przesuwne', 'podstawka pod', 'zbiornik na deszczówkę', 'autko', 'umywalka drewna', 'garnek', 'odświeżacz powietrza', 'transformator dzwonkowy', 'zwierzątko nosidełku', 'świecznik', 'wentylator stołowy', 'ścierka mikrowłókno', 'trzepaczka', 'deska uchwytem', 'wóze

In [35]:
with open('clustered_products.json', 'w') as outfile:
    json.dump(clustered_products_fixed_names, outfile)

z czego wynikały utrudnienia: jest taki produkt jak 'Samochód BMW X6M', ale też 'Samochód elektryczny dla dzieci, biały Mercedes Benz AMG S63', 'Podłokietnik Do Samochodu Vw Polo Ⅳ (2001-2005)' i ciężko odróżnić co jest czym, bo możemy się domyślać, że produkt 'Samochód BMW X6M' jest zabawką

ciekawe produkty: plandeka, markiza, Łącznik krzywkowy, kompostownik ogrodowy, pudlo, stojak kwiat, ekran projekcyjny, sztuczna roslina, klatka dla, meski smoking